In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('..')
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [2]:
import rnacomp
import rnacomp.models
from rnacomp.fastai_fit import *
from rnacomp.dataset import LenMatchBatchSampler, DeviceDataLoader
from rnacomp.utils import seed_everything, MAE, loss_laplace
import gc
from tqdm import tqdm_notebook as tqdm

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
class CFG_test:
    dataset_name ='RNA_Dataset_TestBppSSFullV0' #`RNA_Dataset_TestBppSS' #'
    pathbb = Path("../data/Ribonanza_bpp_files")
    pathss = Path("../eda/test_ss_vienna_rna.parquet")
    bs = 256
    num_workers = 16
    device = 'cuda'
    
    

In [4]:
df_test = pd.read_csv(os.path.join('../data/','test_sequences.csv'))
print(df_test.shape)
fns = list(CFG_test.pathbb.rglob("*.txt"))
bpp_df = pd.DataFrame({"bpp": fns})
bpp_df['sequence_id'] = bpp_df['bpp'].apply(lambda x: x.stem)
print(df_test.shape)
bpp_df.drop_duplicates(subset=['sequence_id'], inplace=True)
ss = pd.read_parquet(CFG_test.pathss)[["sequence_id", "ss_roi", "ss_full"]]
print(df_test.shape)
df_test = pd.merge(df_test, bpp_df, on='sequence_id')
print(df_test.shape)
df_test = pd.merge(df_test, ss, on='sequence_id')
print(df_test.shape)
ds = getattr(rnacomp.dataset, CFG_test.dataset_name)(df_test)
dl = DeviceDataLoader(torch.utils.data.DataLoader(ds, batch_size=CFG_test.bs, 
               shuffle=False, drop_last=False, num_workers=CFG_test.num_workers), CFG_test.device)
del df_test
gc.collect()



(1343823, 5)
(1343823, 5)
(1343823, 5)
(1343823, 6)
(1343823, 8)


0

In [5]:
def get_model_from_cfg(CFG):
    md= getattr(rnacomp.models, CFG.model_name)(dim = CFG.dim)
    mw = f'../exp/{CFG.out}/models/model.pth'
    print(f"loading : {mw}")
    md.load_state_dict(torch.load(mw))
    return md

def generate_sub(config_list, out_name, clip=True):
    models = []
    for m in config_list:
        model = get_model_from_cfg(m)
        model = model.to(CFG_test.device)
        model.eval()
        models.append(model)
    if not clip: 
        out_name = out_name + '_no_clip'
        print(f'generating sub with no cliping: {out_name}')
    ids,preds = [],[]
    for x,y in tqdm(dl):
        with torch.no_grad(),torch.cuda.amp.autocast():
            if clip:
                p = torch.stack([torch.nan_to_num(model(x)) for model in models]
                                ,0).mean(0).clip(0,1)
            else:
                p = torch.stack([torch.nan_to_num(model(x)) for model in models]
                                ,0).mean(0)
                
    
        for idx, mask, pi in zip(y['ids'].cpu(), x['mask'].cpu(), p.cpu()):
            ids.append(idx[mask])
            preds.append(pi[mask[:pi.shape[0]]])

    ids = torch.concat(ids)
    preds = torch.concat(preds)

    df = pd.DataFrame({'id':ids.numpy(), 'reactivity_DMS_MaP':preds[:,1].numpy(), 
                    'reactivity_2A3_MaP':preds[:,0].numpy()})
    df["reactivity_DMS_MaP"] = df["reactivity_DMS_MaP"].astype("float32")
    df["reactivity_2A3_MaP"] = df["reactivity_2A3_MaP"].astype("float32")

    df.to_parquet(f'{out_name}.parquet', index=False) # 6.5GB
    df.head()

In [6]:
# # use this to generate submission
# class CFG:
#     path = Path("../data/")
#     pathbb = Path("../data/Ribonanza_bpp_files")
#     pathss = Path("../eda/train_ss_vienna_rna.parquet")
#     split_id = Path('../eda/fold_split.csv')
#     bs = 64
#     num_workers = 12
#     device = 'cuda'
#     seed = 2023
#     out = 'exp_09_ft'
#     dataset_name = 'RNA_DatasetBaselineSplitbppV0'
#     sn_train = True
    
#     model_name = 'RNA_ModelV7'
#     dim = 192
#     depth = 12
#     dim_head = 32
   
#     epoch = 12
#     lr = 5e-5
#     wd = 0.05
#     pct_start = 0.01
#     md_wt = 'exp_09/models/model.pth'
    
    
# class CFGFT:
#     path = Path("../data/")
#     pathbb = Path("../data/Ribonanza_bpp_files")
#     pathss = Path("../eda/train_ss_vienna_rna.parquet")
#     split_id = Path('../eda/fold_split.csv')
#     bs = 64
#     num_workers = 12
#     device = 'cuda'
#     seed = 2023
#     out = 'exp_08_ft'
#     dataset_name = 'RNA_DatasetBaselineSplitssV0'
#     sn_train = True
    
#     model_name = 'RNA_ModelV3SS'
#     dim = 192
#     depth = 12
#     dim_head = 32
   
#     epoch = 12
#     lr = 5e-5
#     wd = 0.05
#     pct_start = 0.01
#     md_wt = 'exp_08/models/model.pth'

    
    
# out_name = CFG.out + '_' + CFGFT.out
# generate_sub([CFG, CFGFT], out_name, clip=False)
# print(f"kaggle competitions submit stanford-ribonanza-rna-folding -f {out_name}.parquet -m '{out_name}'")

In [7]:
# class CFG12:
#     path = Path("../data/")
#     pathbb = Path("../data/Ribonanza_bpp_files")
#     pathss = Path("../eda/train_ss_vienna_rna.parquet")
#     split_id = Path('../eda/fold_split.csv')
#     bs = 64
#     num_workers = 12
#     device = 'cuda'
#     seed = 2023
#     out = 'exp_12_ft'
#     dataset_name = 'RNA_DatasetBaselineSplitbppV1'
#     sn_train = True
    
#     model_name = 'RNA_ModelV7'
#     dim = 192 * 2
#     depth = 12
#     dim_head = 32
   
#     epoch = 10
#     lr = 5e-5
#     wd = 0.05
#     pct_start = 0.01
    
#     md_wt = 'exp_12/models/model.pth'
    
    
# class CFG14:
#     path = Path("../data/")
#     pathbb = Path("../data/Ribonanza_bpp_files")
#     pathss = Path("../eda/train_ss_vienna_rna.parquet")
#     split_id = Path('../eda/fold_split.csv')
#     bs = 64
#     num_workers = 12
#     device = 'cuda'
#     seed = 2023
#     out = 'exp_14_ft'
#     dataset_name = 'RNA_DatasetBaselineSplitssbppV0'
#     sn_train = True
    
#     model_name = 'RNA_ModelV10'
#     dim = 192 * 2
#     depth = 12
#     dim_head = 32
   
#     epoch = 8
#     lr = 5e-6
#     wd = 0.05
#     pct_start = 0.02
#     md_wt = 'exp_14/models/model.pth'
    
# out_name = CFG12.out + "_" + CFG14.out
# generate_sub([CFG12, CFG14], out_name, clip=False)
# print(f"kaggle competitions submit stanford-ribonanza-rna-folding -f {out_name}.parquet -m '{out_name}'")

loading : ../exp/exp_12_ft/models/model.pth
loading : ../exp/exp_14_ft/models/model.pth
generating sub with no cliping: exp_12_ft_exp_14_ft_no_clip


/tmp/ipykernel_43277/2162199118.py:19: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for x,y in tqdm(dl):


  0%|          | 0/5250 [00:00<?, ?it/s]

kaggle competitions submit stanford-ribonanza-rna-folding -f exp_12_ft_exp_14_ft.parquet -m 'exp_12_ft_exp_14_ft'


In [7]:
class CFG:
    path = Path("../data/")
    pathbb = Path("../data/Ribonanza_bpp_files")
    pathss = Path("../eda/train_ss_vienna_rna.parquet")
    split_id = Path('../eda/fold_split.csv')
    bs = 64
    num_workers = 12
    device = 'cuda'
    seed = 2023
    out = 'exp_16_ft'
    dataset_name = 'RNA_DatasetBaselineSplitssbppV1'
    sn_train = True
    
    model_name = 'RNA_ModelV11'
    dim = 192 * 2
    depth = 12
    dim_head = 32
   
    epoch = 8
    lr = 5e-6
    wd = 0.05
    pct_start = 0.02
    
    md_wt = 'exp_16/models/model.pth'
    
class CFG19:
    path = Path("../data/")
    pathbb = Path("../data/Ribonanza_bpp_files")
    pathss = Path("../eda/train_ss_vienna_rna.parquet")
    split_id = Path('../eda/fold_split.csv')
    bs = 64
    num_workers = 12
    device = 'cuda'
    seed = 2023
    out = 'exp_19_ft'
    dataset_name = 'RNA_DatasetBaselineSplitssbppV1'
    sn_train = True
    
    model_name = 'RNA_ModelV14'
    dim = 192 * 2
    depth = 12
    dim_head = 32
   
    epoch = 8
    lr = 5e-6
    wd = 0.05
    pct_start = 0.02
    
    md_wt = 'exp_19/models/model.pth'
    
out_name = CFG.out + "_" + CFG19.out
generate_sub([CFG, CFG19], out_name, clip=False)
print(f"kaggle competitions submit stanford-ribonanza-rna-folding -f {out_name}.parquet -m '{out_name}'")

loading : ../exp/exp_16_ft/models/model.pth
loading : ../exp/exp_19_ft/models/model.pth
generating sub with no cliping: exp_16_ft_exp_19_ft_no_clip


/tmp/ipykernel_73696/2162199118.py:19: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for x,y in tqdm(dl):


  0%|          | 0/5250 [00:00<?, ?it/s]

In [ ]:
import polars as pl
import matplotlib.pyplot as plt
#read your sub here
df=pl.read_parquet(f"{out_name}.parquet")
#some parameters
font_size=6
id1=269545321
id2=269724007
reshape1=391
reshape2=457
#get predictions
pred_DMS=df[id1:id2+1]['reactivity_DMS_MaP'].to_numpy().reshape(reshape1,reshape2)
pred_2A3=df[id1:id2+1]['reactivity_2A3_MaP'].to_numpy().reshape(reshape1,reshape2)
#plot mutate and map
fig = plt.figure()
plt.subplot(121)
plt.title(f'reactivity_DMS_MaP', fontsize=font_size)
plt.imshow(pred_DMS,vmin=0,vmax=1, cmap='gray_r')
plt.subplot(122)
plt.title(f'reactivity_2A3_MaP', fontsize=font_size)
plt.imshow(pred_2A3,vmin=0,vmax=1, cmap='gray_r')
plt.tight_layout()
plt.savefig(f"plot.png",dpi=500)
plt.clf()
plt.close()


In [11]:
pd.read_parquet('exp_12_ft_exp_14_ft.parquet')

,id,reactivity_DMS_MaP,reactivity_2A3_MaP
0,0,0.016083,0.029724
1,1,0.000437,0.041595
2,2,0.000960,0.092957
3,3,0.876953,0.760742
4,4,0.995605,0.501465
...,...,...,...
269796666,269796666,0.203125,0.065125
269796667,269796667,0.241455,0.053101
269796668,269796668,0.209351,0.065491
269796669,269796669,0.272705,0.123047


In [10]:
pd.read_parquet('exp_12_ft_exp_14_ft_no_clip.parquet')

,id,reactivity_DMS_MaP,reactivity_2A3_MaP
0,0,0.016052,0.029724
1,1,0.000444,0.041595
2,2,0.000968,0.093018
3,3,0.876953,0.760742
4,4,0.996094,0.501953
...,...,...,...
269796666,269796666,0.203125,0.065125
269796667,269796667,0.241455,0.053101
269796668,269796668,0.209351,0.065491
269796669,269796669,0.272705,0.123047
